https://huggingface.co/Qwen/Qwen2.5-7B-Instruct

https://www.tensorflow.org/install/pip

In [1]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-n2qum5cf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-n2qum5cf
  Resolved https://github.com/huggingface/transformers to commit fa3f2db5c7405a742fcb8f686d3754f70db00977
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.5 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9972254 sha256=3e0aa2a969440224d79218a36075dd62b722983fe1f54fefb64d293054e725c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-cjs7q0wk/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully u

In [2]:
!pip install accelerate

In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

device = accelerator.device
model, optimizer, data = accelerator.prepare(model, optimizer, data)
accelerator.backward(loss)

In [ ]:
import gc
gc.collect()

0

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
import gc
gc.collect()
accelerator = Accelerator()

device = accelerator.device

model_name = "Qwen/Qwen2.5-7B-Instruct"
#model, optimizer, data = accelerator.prepare(model_name, optimizer, data)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
import gc
gc.collect()
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
import gc
gc.collect()
accelerator.backward(loss)
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from accelerate import Accelerator
import torch
import gc

gc.collect()
accelerator = Accelerator()

device = accelerator.device

model_name = "Qwen/Qwen2.5-7B-Instruct"

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create an optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)  # You can adjust the learning rate

# Create a sample dataset (replace with your actual data)
data = torch.randint(0, tokenizer.vocab_size, (16, 128))

# Prepare the model, optimizer, and data with the accelerator
model, optimizer, data = accelerator.prepare(model, optimizer, data)
model = torch.nn.Transformer()
# Rest of your code...
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
import gc
gc.collect()
# Define your loss function and calculate the loss (replace with your actual loss calculation)
# Example:
# loss = some_loss_function(model_outputs, targets)

# Backpropagate the loss using the accelerator
# accelerator.backward(loss)
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
!accelerate config -h

usage: accelerate <command> [<args>] config [-h] [--config_file CONFIG_FILE] {default,update} ...

Launches a series of prompts to create and save a `default_config.yaml` configuration file for
your training system. Should always be ran first on your machine

options:
  -h, --help            show this help message and exit
  --config_file CONFIG_FILE, --config-file CONFIG_FILE
                        The path to use to store the config file. Will default to a file named
                        default_config.yaml in the cache location, which is the content of the
                        environment `HF_HOME` suffixed with 'accelerate', or if you don't have
                        such an environment variable, your cache directory ('~/.cache' or the
                        content of `XDG_CACHE_HOME`) suffixed with 'huggingface'.

subcommands:
  {default,update}
    default             Create a default config file for Accelerate with only a few flags set.
    update              Update 

In [ ]:
!accelerate config /content/single_gpu.yaml

In [ ]:
!accelerate launch --config_file /content/single_gpu.yaml

usage: accelerate <command> [<args>] launch [-h] [--config_file CONFIG_FILE] [--quiet] [--cpu]
                                            [--multi_gpu] [--tpu] [--ipex]
                                            [--mixed_precision {no,fp16,bf16,fp8}]
                                            [--num_processes NUM_PROCESSES]
                                            [--num_machines NUM_MACHINES]
                                            [--num_cpu_threads_per_process NUM_CPU_THREADS_PER_PROCESS]
                                            [--enable_cpu_affinity]
                                            [--dynamo_backend {no,eager,aot_eager,inductor,aot_ts_nvfuser,nvprims_nvfuser,cudagraphs,ofi,fx2trt,onnxrt,tensorrt,aot_torchxla_trace_once,torhchxla_trace_once,ipex,tvm}]
                                            [--dynamo_mode {default,reduce-overhead,max-autotune}]
                                            [--dynamo_use_fullgraph] [--dynamo_use_dynamic]
                  

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!accelerate env


Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.34.2
- Platform: Linux-6.1.85+-x86_64-with-glibc2.35
- `accelerate` bash location: /usr/local/bin/accelerate
- Python version: 3.10.12
- Numpy version: 1.26.4
- PyTorch version (GPU?): 2.4.1+cu121 (True)
- PyTorch XPU available: False
- PyTorch NPU available: False
- PyTorch MLU available: False
- PyTorch MUSA available: False
- System RAM: 12.67 GB
- GPU type: Tesla T4
- `Accelerate` default config:
	Not found


In [4]:
!git clone https://github.com/huggingface/accelerate.git

Cloning into 'accelerate'...
remote: Enumerating objects: 14731, done.
remote: Counting objects: 100% (1318/1318), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 14731 (delta 1279), reused 1235 (delta 1234), pack-reused 13413 (from 1)
Receiving objects: 100% (14731/14731), 4.80 MiB | 10.62 MiB/s, done.
Resolving deltas: 100% (10269/10269), done.


In [ ]:
%cd /content/accelerate/examples

In [ ]:
!pip install 'accelerate>=0.27.0' 'torchpippy>=0.2.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
!accelerate launch --num_processes 1 /content/accelerate/examples/inference/distributed/phi2.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading checkpoint shards:   0% 0/2 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/content/accelerate/examples/inference/distributed/phi2.py", line 27, in <module>
    model = AutoModelForCausalLM.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 4191, in from_pretrained
    ) = cls._load_pretrained_model(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 4694, in _load_pretrained_model
    new_e

In [ ]:
!accelerate launch --num_machines=1 --mixed_precision=fp16 --dynamo_backend=no /content/accelerate/examples/inference/distributed/phi2.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading checkpoint shards:   0% 0/2 [00:01<?, ?it/s]
Traceback (most recent call last):
  File "/content/accelerate/examples/inference/distributed/phi2.py", line 27, in <module>
    model = AutoModelForCausalLM.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 4191, in from_pretrained
    ) = cls._load_pretrained_model(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 4694, in _load_pretrained_model
    new_error_msgs, offload_index, state_dict_index = _load_state_dict_into_meta_model(
  File "/usr/local/

In [ ]:
!accelerate launch --config_file default_config.yaml /content/accelerate/examples/inference/distributed/phi2.py

Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/accelerate_cli.py", line 48, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 1152, in launch_command
    args, defaults, mp_from_config_flag = _validate_launch_command(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 986, in _validate_launch_command
    defaults = load_config_from_file(args.config_file)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/config/config_args.py", line 46, in load_config_from_file
    raise FileNotFoundError(
FileNotFoundError: The passed configuration file `default_config.yaml` does not exist. Please pass an existing file to `accelerate launch`, or use the default one created through `accelerate config` and run `accelerate launch` without the `--config_file

In [ ]:
!accelerate launch --config_file default_config.yaml /content/accelerate/examples/inference/distributed/phi2.py

Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/accelerate_cli.py", line 48, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 1152, in launch_command
    args, defaults, mp_from_config_flag = _validate_launch_command(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 986, in _validate_launch_command
    defaults = load_config_from_file(args.config_file)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/config/config_args.py", line 46, in load_config_from_file
    raise FileNotFoundError(
FileNotFoundError: The passed configuration file `default_config.yaml` does not exist. Please pass an existing file to `accelerate launch`, or use the default one created through `accelerate config` and run `accelerate launch` without the `--config_file

In [ ]:
!accelerate config

----------------------------------------------------------------------------------------------------In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)
This machine
This machine
----------------------------------------------------------------------------------------------------Which type of machine are you using?
Please input a choice index (starting from 0), and press enter
 ➔  No distributed training
    multi-CPU
    multi-XPU
    multi-GPU
    multi-NPU
    multi-MLU
    multi-MUSA
    TPU
TPU
No distributed training
Do you want to run your training on CPU only (even if a GPU / Apple Silicon / Ascend NPU device is available)? [yes/NO]:yes
Do you want to use Intel PyTorch Extension (IPEX) to speed up training on CPU? [yes/NO]:yes
Do you wish to optimize your script with torch dynamo?[yes/NO]:yes
-----------------------------------------------------------------------------------------

In [ ]:
!accelerate launch /content/single_gpu.yaml /content/accelerate/examples/inference/distributed/phi2.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.


In [3]:
!pip3 install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.2-py3-none-any.whl size=1521840 sha256=498f39bcf190fc6ab82c4837dda28508e11fa74344939fe8ee969eb4fed1c3f2
  Stored in directory: /root/.cache/pip/wheels/53/f1/46/5d54ff3c9f0af8cab280e54887af31c3a6b489b409ba8e3f5c
Successfully built deepspeed


### شغال

In [5]:
!accelerate launch --num_processes 1 phi2.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
config.json: 100% 735/735 [00:00<00:00, 3.69MB/s]
model.safetensors.index.json: 100% 35.7k/35.7k [00:00<00:00, 83.5MB/s]
model-00001-of-00002.safetensors:   0% 0.00/5.00G [00:00<?, ?B/s]
model-00001-of-00002.safetensors:   0% 10.5M/5.00G [00:00<03:35, 23.2MB/s]
model-00001-of-00002.safetensors:   0% 21.0M/5.00G [00:00<03:30, 23.6MB/s]
model-00001-of-00002.safetensors:   1% 31.5M/5.00G [00:01<03:27, 23.9MB/s]
model-00001-of-00002.safetensors:   1% 41.9M/5.00G [00:01<03:25, 24.1MB/s]
model-00001-of-00002.safetensors:   1% 52.4M/5.00G [00:02<03:24, 24.2MB/s]
model-00001-of-00002.safetensors:   1% 62.9M/5.00G [00:02<03:23, 24.2MB/s]
model-00001-of-00002.safe

In [ ]:
import gc
import torch

# إفراغ ذاكرة وحدة معالجة الرسومات
torch.cuda.empty_cache()

# إفراغ ذاكرة الوصول العشوائي
gc.collect()

# إفراغ ذاكرة القرص (مثال على حذف ملف)
!rm -rf /path/to/file

### شغال

In [6]:
!accelerate launch --num_processes 1 phi2.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading checkpoint shards: 100% 2/2 [00:14<00:00,  7.34s/it]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
['Who is Napoleon Bonaparte?\n\nNapoleon Bonaparte was a French military and political leader who rose to prominence during the']


### شغال

In [9]:
!accelerate launch --num_processes 1 phi2.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
config.json: 100% 601/601 [00:00<00:00, 4.19MB/s]
model.safetensors.index.json: 100% 23.9k/23.9k [00:00<00:00, 77.9MB/s]
model-00001-of-00003.safetensors:   0% 0.00/4.95G [00:00<?, ?B/s]
model-00001-of-00003.safetensors:   1% 31.5M/4.95G [00:00<00:18, 261MB/s]
model-00001-of-00003.safetensors:   2% 83.9M/4.95G [00:00<00:13, 354MB/s]
model-00001-of-00003.safetensors:   3% 126M/4.95G [00:00<00:13, 364MB/s] 
model-00001-of-00003.safetensors:   3% 168M/4.95G [00:00<00:13, 344MB/s]
model-00001-of-00003.safetensors:   4% 210M/4.95G [00:00<00:14, 321MB/s]
model-00001-of-00003.safetensors:   5% 252M/4.95G [00:00<00:14, 329MB/s]
model-00001-of-00003.safetensors: 

In [ ]:
!huggingface-cli login

In [10]:
!accelerate launch --num_processes 1 phi2.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading checkpoint shards: 100% 3/3 [01:03<00:00, 21.08s/it]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
['Who is Napoleon Bonaparte?\n\nNapoleon Bonaparte was a French military and political leader who rose to prominence during the French Revolution and its associated wars. He was born on August 15, 1769, on the island of Corsica, which was then a possession of the Republic of Genoa. Napoleon\'s father, Carlo Buonaparte, was a lawyer and a member of the Corsican nobility, while his mother, Letizia Ramolino, was a woman of humble origins.\n\nNapoleon was educated at a Jesuit school in Ajaccio, the capital of Corsica, and later 

In [ ]:
accelerate/examples/inference/distributed
/phi2.py
# Copyright 2024 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from accelerate import PartialState
from accelerate.utils import gather_object


# Start up the distributed environment without needing the Accelerator.
distributed_state = PartialState()

# You can change the model to any LLM such as mistralai/Mistral-7B-v0.1 or meta-llama/Llama-2-7b-chat-hf
model_name = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map=distributed_state.device, torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
# Need to set the padding token to the eos token for generation
tokenizer.pad_token = tokenizer.eos_token

prompts = [
    "I would like to",
    "hello how are you",
    "what is going on",
    "roses are red and",
    "welcome to the hotel",
]

# You can change the batch size depending on your GPU RAM
batch_size = 2
# We set it to 8 since it is better for some hardware. More information here https://github.com/huggingface/tokenizers/issues/991
pad_to_multiple_of = 8

# Split into batches
# We will get the following results:
# [ ["I would like to", "hello how are you"], [ "what is going on", "roses are red and"], [ "welcome to the hotel"] ]
formatted_prompts = [prompts[i : i + batch_size] for i in range(0, len(prompts), batch_size)]

# Apply padding on the left since we are doing generation
padding_side_default = tokenizer.padding_side
tokenizer.padding_side = "left"
# Tokenize each batch
tokenized_prompts = [
    tokenizer(formatted_prompt, padding=True, pad_to_multiple_of=pad_to_multiple_of, return_tensors="pt")
    for formatted_prompt in formatted_prompts
]
# Put back the original padding behavior
tokenizer.padding_side = padding_side_default

completions_per_process = []
# We automatically split the batched data we passed to it across all the processes. We also set apply_padding=True
# so that the GPUs will have the same number of prompts, and you can then gather the results.
# For example, if we have 2 gpus, the distribution will be:
# GPU 0: ["I would like to", "hello how are you"],  "what is going on", "roses are red and"]
# GPU 1: ["welcome to the hotel"], ["welcome to the hotel"] -> this prompt is duplicated to ensure that all gpus have the same number of prompts
with distributed_state.split_between_processes(tokenized_prompts, apply_padding=True) as batched_prompts:
    for batch in batched_prompts:
        # Move the batch to the device
        batch = batch.to(distributed_state.device)
        # We generate the text, decode it and add it to the list completions_per_process
        outputs = model.generate(**batch, max_new_tokens=20)
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        completions_per_process.extend(generated_text)

# We are gathering string, so we need to use gather_object.
# If you need to gather tensors, you can use gather from accelerate.utils
completions_gather = gather_object(completions_per_process)

# Drop duplicates produced by apply_padding in split_between_processes
completions = completions_gather[: len(prompts)]

distributed_state.print(completions)

Who is Napoleon Bonaparte?\n\nNapoleon Bonaparte was a French military and political leader who rose to prominence during the French Revolution and its associated wars. He was born on August 15, 1769, on the island of Corsica, which was then a possession of the Republic of Genoa. Napoleon\'s father, Carlo Buonaparte, was a lawyer and a member of the Corsican nobility, while his mother, Letizia Ramolino, was a woman of humble origins.\n\nNapoleon was educated at a Jesuit school in Ajaccio, the capital of Corsica, and later at the Royal Military Academy in Brienne-le-Château, France. He was commissioned as a second lieutenant in the French Army in 1785, and served in various capacities during the French Revolution.\n\nIn 1796, Napoleon was given command of the French Army in Italy, where he quickly established a reputation as a brilliant military strategist. He defeated the Austrian forces and expanded French control over much of Italy, earning the nickname "the Little Corporal." In 1799, he staged a coup and took control of the French government, becoming First Consul of the French Republic.\n\nAs First Consul, Napoleon consolidated his power and began a series of military campaigns that would make him one of the most powerful men in Europe. He invaded Egypt in 1798, establishing a French presence in the Middle East and challenging British control of the region. He also invaded Switzerland, Austria, and Russia, and defeated the British at the Battle of Trafalgar in 1805.\n\nIn 1804, Napoleon crowned himself Emperor of the French, and he ruled as such until his abdication in 1814. During his reign, he implemented a series of political and social reforms, including the creation of a modern legal code, the Code Napoléon, and the promotion of education and economic development.\n\nNapoleon was exiled to the island of Elba in 1814, but he escaped and returned to France in 1815. He was defeated at the Battle of Waterloo in June of that year and was exiled to the island of Saint Helena, where he died on May 5, 1821.\n\nNapoleon Bonaparte is remembered as one of the most important figures in modern European history. He is credited with transforming France and Europe through his military victories, political reforms, and cultural achievements. He is also known for his ambition, charisma, and intelligence, as well as his ruthless and authoritarian rule.']

In [ ]:
/content/a.py
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [ ]:
!accelerate launch --num_processes 1 a.py

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [12]:
!pip install tensorflow-gpu==2.13.0  # Or your desired TF version
!pip install nvidia-tensorrt

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.13.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==2.13.0
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-10.5.0-py2.py3-none-any.whl size=16338 sha256=63451f63e0c59726f7870b3876c56ad45c462fa344feb628f5182fc81c51ea72
  Stored in directory: /root/.cache/pip/wheels/17/93/2c/4db84c8fffd36bab9c54b39b4b886aaaaa1a1ab6d094d28a86
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.5.0-py2.py3-none-any.whl size=17556 sha256=1d0e090d7e4aab72afbcec08710091630960389634e7adfed2b6d51c8b699a2c
  Stored in directory: /root/.cache/pip/wheels/31/83/8a/84edbd0d600c1b334a5ac98e18626a458dc8a7

In [14]:
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!nvidia-smi

In [17]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
import tensorflow as tf
   print(tf.__version__)

شغال

In [ ]:
!pip install git+https://github.com/huggingface/transformers

شغال

In [ ]:
!pip install -r /content/requirements-docs.txt

شغال

In [ ]:
!python3 -m pip install tensorflow[and-cuda]
# Verify the installation:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

### شغال

In [8]:
!accelerate launch --num_processes 1 aa.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading checkpoint shards: 100% 4/4 [01:00<00:00, 15.01s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
2024-10-14 20:29:02.228082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 20:29:02.509538: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 20:29:02.594712: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable t

In [ ]:
Who is Napoleon Bonaparte?

In [9]:
!accelerate launch --num_processes 1 aaa.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading checkpoint shards: 100% 4/4 [01:01<00:00, 15.40s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
2024-10-14 20:39:51.271209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 20:39:51.548291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 20:39:51.630338: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable t

In [ ]:
/content/phi2.py======mistralv3
/content/aaa.py====Qwen2.5-7B-Instruct


hالاكواد التى اشتغلت جيدا
/content/phi2.py
/content/aaa.py

In [ ]:
phi2.py
# Copyright 2024 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from accelerate import PartialState
from accelerate.utils import gather_object


# Start up the distributed environment without needing the Accelerator.
distributed_state = PartialState()

# You can change the model to any LLM such as mistralai/Mistral-7B-v0.1 or meta-llama/Llama-2-7b-chat-hf
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map=distributed_state.device, torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
# Need to set the padding token to the eos token for generation
tokenizer.pad_token = tokenizer.eos_token

prompts = [
    "Who is Napoleon Bonaparte?",
]

# You can change the batch size depending on your GPU RAM
batch_size = 2
# We set it to 8 since it is better for some hardware. More information here https://github.com/huggingface/tokenizers/issues/991
pad_to_multiple_of = 8

# Split into batches
# We will get the following results:
# [ ["I would like to", "hello how are you"], [ "what is going on", "roses are red and"], [ "welcome to the hotel"] ]
formatted_prompts = [prompts[i : i + batch_size] for i in range(0, len(prompts), batch_size)]

# Apply padding on the left since we are doing generation
padding_side_default = tokenizer.padding_side
tokenizer.padding_side = "left"
# Tokenize each batch
tokenized_prompts = [
    tokenizer(formatted_prompt, padding=True, pad_to_multiple_of=pad_to_multiple_of, return_tensors="pt")
    for formatted_prompt in formatted_prompts
]
# Put back the original padding behavior
tokenizer.padding_side = padding_side_default

completions_per_process = []
# We automatically split the batched data we passed to it across all the processes. We also set apply_padding=True
# so that the GPUs will have the same number of prompts, and you can then gather the results.
# For example, if we have 2 gpus, the distribution will be:
# GPU 0: ["I would like to", "hello how are you"],  "what is going on", "roses are red and"]
# GPU 1: ["welcome to the hotel"], ["welcome to the hotel"] -> this prompt is duplicated to ensure that all gpus have the same number of prompts
with distributed_state.split_between_processes(tokenized_prompts, apply_padding=True) as batched_prompts:
    for batch in batched_prompts:
        # Move the batch to the device
        batch = batch.to(distributed_state.device)
        # We generate the text, decode it and add it to the list completions_per_process
        outputs = model.generate(**batch, max_new_tokens=1024)
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        completions_per_process.extend(generated_text)

# We are gathering string, so we need to use gather_object.
# If you need to gather tensors, you can use gather from accelerate.utils
completions_gather = gather_object(completions_per_process)

# Drop duplicates produced by apply_padding in split_between_processes
completions = completions_gather[: len(prompts)]

distributed_state.print(completions)


In [ ]:
/content/aaa.py
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
model_name = "Qwen/Qwen2.5-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",          # Automatically choose float32 or float16 based on device
    device_map="auto"            # Automatically map layers to available devices (e.g., GPU)
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define prompt and messages
prompt = "Who is Napoleon Bonaparte?"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

# Prepare inputs for the model
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Generate output from the model
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

# Post-process generated output
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Print the response from the model
print(response)


In [ ]:
/content/aa.py
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
model_name = "Qwen/Qwen2.5-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",          # Automatically choose float32 or float16 based on device
    device_map="auto"            # Automatically map layers to available devices (e.g., GPU)
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define prompt and messages
prompt = "Give me a short introduction to large language models."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

# Prepare inputs for the model
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Generate output from the model
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

# Post-process generated output
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Print the response from the model
print(response)


In [ ]:
/content/requirements-docs.txt
furo
myst-parser==4.0.0
sphinx<8
sphinx-copybutton
sphinx-design>=0.6.0

In [ ]:

https://huggingface.co/Qwen/Qwen2.5-7B-Instruct

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
